# EARTH ENGINE PYTHON API TO POSTGRESQL

### Inicializar a conexão com a a plataforma do Earth Engine

In [6]:
import ee
print("Iniciando conexão com EE...");
ee.Initialize();
print("Conectado!");

Iniciando conexão com EE...
Conectado!


#### Região de estudo

In [24]:
braco = 'ocoi'

In [25]:
regiao = ee.Feature(ee.FeatureCollection("users/odraitaipu/bracos/"+braco).union().first()).set('nome',braco)

#### Coleção
Escolha do sensor<br>
Filtrando apenas na região de interesse com até 5% de nuvem


In [27]:
sensor = 'COPERNICUS/S2'   # Sentinel 2 
nuvens = 5                 # Máximo de cobertuda de nuvens
bandas = ['B8','B4','B11'] # Selecionar apenas as bandas necessárias, para o calculo de de ndvi e exclusão do bloom de algas.

In [22]:
collection = (ee.ImageCollection(sensor)
              .filterBounds(regiao.geometry())
              .filter(ee.Filter.lessThanOrEquals('CLOUDY_PIXEL_PERCENTAGE',nuvens))
              .select(bandas)
              .sort('system:time_start'))

#### Função que transoforma uma imagem de satélite multiband em uma imagem 'macrofita', binária, onde 1 indica presença e 0 ausência de macrófitas


In [23]:
def getMacrofitasSentinel(image):

	image = ee.Image(image).divide(10000);

	bands = image.bandNames();
	ndvi = image.normalizedDifference();

	soma = ndvi.reproject(image.select('B4').projection()).reduceResolution(ee.Reducer.sum());
	porcentagem = soma.divide(ndvi).multiply(100).round();
	clippedNdvi = ndvi.updateMask(ndvi.where(porcentagem.lte(99),0));

	output = ee.Image(0).where(clippedNdvi.gte(0.2),1);
	result = output.updateMask(output).rename(['macrofitas']).reproject(image.select('B4').projection());

	return result;